In [63]:
import psycopg2
import pandas as pd

# Conecta com o banco de dados

In [64]:
conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='1001',
    host='localhost',
    port='5432'
)

# Cria o cursor

In [65]:

cursor = conn.cursor()

# Cria tabela no db

In [66]:
# Cria tabela no db
try:
    cursor.execute("""
                
-- Drop existing tables if they exist (optional, for clean runs)
-- Use with caution, this will delete all data!
DROP TABLE IF EXISTS public."Country" CASCADE;
DROP TABLE IF EXISTS public."Survey" CASCADE;
DROP TABLE IF EXISTS public."Region" CASCADE; -- Typo matches schema
DROP TABLE IF EXISTS public."Demography" CASCADE;
DROP TABLE IF EXISTS public."Health" CASCADE;
DROP TABLE IF EXISTS public."Education" CASCADE;
DROP TABLE IF EXISTS public."Economy" CASCADE;
DROP TABLE IF EXISTS public."Population" CASCADE;
DROP TABLE IF EXISTS public."Employment" CASCADE;
DROP TABLE IF EXISTS public."Life Expectancy" CASCADE;
DROP TABLE IF EXISTS public."Decile" CASCADE;

CREATE TABLE IF NOT EXISTS public."Country"
(
    country_code character varying NOT NULL,
    country_name character varying,
    region_code character varying NOT NULL,
    PRIMARY KEY (country_code)
);

CREATE TABLE IF NOT EXISTS public."Survey"
(
    id_survey serial NOT NULL,
    country_code character varying NOT NULL,
    welfare_type character varying,
    survey_acronym character varying,
    survey_comparability integer,
    comparable_spell character varying,
    poverty_line double precision,
    headcount double precision,
    poverty_gap double precision,
    poverty_severity double precision,
    gini double precision,
    reporting_pop integer,
    reporting_pce double precision,
    distribution_type character varying,
    spl double precision,
    survey_year integer,
    PRIMARY KEY (id_survey)
);

CREATE TABLE IF NOT EXISTS public."Region"
(
    region_code character varying NOT NULL,
    region_name character varying NOT NULL,
    PRIMARY KEY (region_code)
);

CREATE TABLE IF NOT EXISTS public."Decile"
(
    id_decile serial NOT NULL,
    value double precision,
    name character varying,
    id_survey integer NOT NULL,
    PRIMARY KEY (id_decile)
);

CREATE TABLE IF NOT EXISTS public."Demography"
(
    id_demography serial NOT NULL,
    year integer,
    country_code character varying NOT NULL,
    pop_density double precision,
    urban_pop double precision,
    rural_pop double precision,
    net_migration double precision,
    death_rate double precision,
    birth_rate double precision,
    PRIMARY KEY (id_demography)
);

CREATE TABLE IF NOT EXISTS public."Employment"
(
    id_employment serial NOT NULL,
    year integer,
    country_code character varying NOT NULL,
    child_emp double precision,
    unemp double precision,
    vulnerable_emp double precision,
    part_time double precision,
    employers double precision,
    labor_force_total double precision,
    labor_force_fem double precision,
    PRIMARY KEY (id_employment)
);

CREATE TABLE IF NOT EXISTS public."Education"
(
    id_education serial NOT NULL,
    year integer,
    country_code character varying NOT NULL,
    child_out_of_school double precision,
    progression_to_sec double precision,
    expenditure double precision,
    preprim_enroll double precision,
    prim_enrol double precision,
    sec_enrol double precision,
    terti_enrol double precision,
    PRIMARY KEY (id_education)
);

CREATE TABLE IF NOT EXISTS public."Economy"
(
    id_economy serial NOT NULL,
    year integer,
    country_code character varying NOT NULL,
    gdp double precision,
    inflation double precision,
    tax_revenue double precision,
    PRIMARY KEY (id_economy)
);

CREATE TABLE IF NOT EXISTS public."Population"
(
    id_population serial NOT NULL,
    id_demography integer NOT NULL,
    pop_ages character varying,
    gender character varying,
    "number" double precision,
    PRIMARY KEY (id_population)
);

CREATE TABLE IF NOT EXISTS public."Life Expectancy"
(
    id_life_expectancy serial NOT NULL,
    id_demography integer NOT NULL,
    gender character varying,
    value double precision,
    PRIMARY KEY (id_life_expectancy)
);

CREATE TABLE IF NOT EXISTS public."Health"
(
    id_health serial NOT NULL,
    country_code character varying NOT NULL,
    year integer,
    hospital_beds double precision,
    physicians double precision,
    expenditure double precision,
    PRIMARY KEY (id_health)
);

ALTER TABLE IF EXISTS public."Country"
    ADD FOREIGN KEY (region_code)
    REFERENCES public."Region" (region_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Survey"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Decile"
    ADD FOREIGN KEY (id_survey)
    REFERENCES public."Survey" (id_survey) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Demography"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Employment"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Education"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Economy"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Population"
    ADD FOREIGN KEY (id_demography)
    REFERENCES public."Demography" (id_demography) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Life Expectancy"
    ADD FOREIGN KEY (id_demography)
    REFERENCES public."Demography" (id_demography) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;


ALTER TABLE IF EXISTS public."Health"
    ADD FOREIGN KEY (country_code)
    REFERENCES public."Country" (country_code) MATCH SIMPLE
    ON UPDATE NO ACTION
    ON DELETE NO ACTION
    NOT VALID;
    """)

    conn.commit()
    print("Tabela recriada do zero!")

except Exception as e:
    conn.rollback()
    print(f"Erro: {e}") 

Tabela recriada do zero!


# Reativar verificações

In [67]:
cursor.execute("SET session_replication_role = 'origin';")
conn.commit()

# Lê o CSV e separa os dados

In [68]:
df_poverty = pd.read_csv('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/poverty_inequality/Poverty_Inequality_filtered.csv')
df_indicators = pd.read_csv('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/global_indicators/Global_Indicators_filtered.csv')

df_region = pd.read_csv('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/poverty_inequality/entities_csv/Region.csv')
df_country = pd.read_csv('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/poverty_inequality/entities_csv/Country.csv')

# Divide o DataFrame conforme alguma condição ou seleção de colunas

In [69]:

df_survey = df_poverty[['country_code', 'welfare_type', 'survey_acronym', 'survey_comparability', 
                'comparable_spell', 'poverty_line', 'headcount', 'poverty_gap', 
                'poverty_severity', 'gini', 'reporting_pop', 'reporting_pce', 
                'distribution_type', 'spl', 'survey_year']]  # Para a tabela 'Survey'


# Passo 1: Carregar dados na tabela 'Region'

In [70]:

# Verificar duplicidade e só inserir valores únicos
df_region.drop_duplicates(subset=['region_code'], inplace=True)  # Remover duplicatas por 'region_code'


with open('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/poverty_inequality/entities_csv/Region.csv', 'r') as f:
    cursor.copy_expert("""
        COPY public."Region" (region_code, region_name)
        FROM STDIN
        WITH (FORMAT csv, HEADER true, DELIMITER ',');
    """, f)
conn.commit()
print("Tabela Region populada com sucesso!")

"""
# 7. Testar a consulta CORRETAMENTE (usando aspas)
cursor.execute('SELECT * FROM public."Region"')
rows = cursor.fetchall()
print(f"\nTotal de registros: {len(rows)}")
for row in rows:
    print(row)
"""

Tabela Region populada com sucesso!


'\n# 7. Testar a consulta CORRETAMENTE (usando aspas)\ncursor.execute(\'SELECT * FROM public."Region"\')\nrows = cursor.fetchall()\nprint(f"\nTotal de registros: {len(rows)}")\nfor row in rows:\n    print(row)\n'

# Passo 2: Carregar dados na tabela 'Country'

In [71]:

# Verificar duplicidade e só inserir valores únicos

with open('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/processing/poverty_inequality/entities_csv/Country.csv', 'r') as f1: # Pode dar erro com duplicatas
    cursor.copy_expert("""
        COPY public."Country" (region_code, country_name, country_code)
        FROM STDIN
        WITH (FORMAT csv, HEADER true, DELIMITER ',');
    """, f1)

# Passo 3: Carregar dados na tabela 'Survey'

In [72]:

with open('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/Poverty_Inequality_filtered_survey.csv', 'w') as f2:
    df_survey.to_csv(f2, index=False)

with open('/home/victor/Documents/MC356/Postgres-Database-Project-PovertyStudy/Poverty_Inequality_filtered_survey.csv', 'r') as f2:
    cursor.copy_expert("""
        COPY public."Survey" (country_code, welfare_type, survey_acronym, survey_comparability, 
                               comparable_spell, poverty_line, headcount, poverty_gap, poverty_severity, 
                               gini, reporting_pop, reporting_pce, distribution_type, spl, survey_year)
        FROM STDIN
        WITH (FORMAT csv, HEADER true, DELIMITER ',')
    """, f2)


In [73]:
# Query de INSERT na tabela REGION
insert_query = """
    INSERT INTO Region (region_code, region_name)
    VALUES (%s, %s)
"""
dados = ("REG", "Pedro")

try:
    cursor.execute(insert_query, dados)
    conn.commit()  # Confirma a transação
    print("Dados inseridos com sucesso!")
except Exception as e:
    conn.rollback()  # Desfaz em caso de erro
    print(f"Erro ao inserir: {e}")


Erro ao inserir: relation "region" does not exist
LINE 2:     INSERT INTO Region (region_code, region_name)
                        ^



# Teste

In [74]:
cursor.execute("""
    SELECT * FROM "Region"
""")
rows = cursor.fetchall()
for row in rows:
    print(row)

('SSA', 'Sub-Saharan Africa')
('ECA', 'Europe & Central Asia')
('OHI', 'Other High Income Countries')
('LAC', 'Latin America & Caribbean')
('SAS', 'South Asia')
('EAP', 'East Asia & Pacific')
('MNA', 'Middle East & North Africa')


# Commit das inserções

In [75]:

conn.commit()

# Fechar a conexão

In [76]:

cursor.close()
conn.close()